In [19]:
import sys
import torch
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

sns.set_theme(style="white")

PATH_TO_GDSC_SCREENING_DATA = '../../datasets/gdsc/screening_data/'
PATH_TO_SAVE_DATA_TO = '../../datasets/gdsc/my_datasets/'
GDSC1_IC50_FILE = 'GDSC1_fitted_dose_response_25Feb20.xlsx'
GDSC2_IC50_FILE = 'GDSC2_fitted_dose_response_25Feb20.xlsx'

GDSC1_RAW_FILE = 'GDSC1_public_raw_data_25Feb20.csv'
GDSC2_RAW_FILE = 'GDSC2_public_raw_data_25Feb20.csv'

CELL_LINE_DETAILS_FILE = 'Cell_Lines_Details.xlsx'

---

# Build GDSC Base Table

Build a GDSC basetable which to use for further analysis

In [3]:
# Read the IC50 files.

# GDSC1
start = time.time()
gdsc1_ic50s = pd.read_excel(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC1_IC50_FILE}', header=0)
print(f"File `{GDSC1_IC50_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc1_ic50s.shape}")

# GDSC2
start = time.time()
gdsc2_ic50s = pd.read_excel(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC2_IC50_FILE}', header=0)
print(f"File `{GDSC2_IC50_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc2_ic50s.shape}")

File `GDSC1_fitted_dose_response_25Feb20.xlsx` took 82.30760 seconds to import. It has shape (310904, 19)
File `GDSC2_fitted_dose_response_25Feb20.xlsx` took 35.37953 seconds to import. It has shape (135242, 19)


In [6]:
# Join both datasets for analysis purposes.
gdsc_ic50s_join = pd.concat([gdsc1_ic50s, gdsc2_ic50s], ignore_index=True)
print(f"Shape: {gdsc_ic50s_join.shape}")

assert gdsc_ic50s_join[gdsc_ic50s_join.index.duplicated()].shape[0] == 0
assert gdsc_ic50s_join.shape[0] == gdsc1_ic50s.shape[0] + gdsc2_ic50s.shape[0]

Shape: (446146, 19)


In [10]:
# Read the raw files.

# GDSC1
start = time.time()
gdsc1_raw = pd.read_csv(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC1_RAW_FILE}', header=0)
print(f"File `{GDSC1_RAW_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc1_raw.shape}")

# GDSC2
start = time.time()
gdsc2_raw = pd.read_csv(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC2_RAW_FILE}', header=0)
print(f"File `{GDSC2_RAW_FILE}` took {time.time()-start:.5f} seconds to import. It has shape {gdsc2_raw.shape}")

# Join both datasets for analysis purposes.
gdsc_raw_join = pd.concat([gdsc1_raw, gdsc2_raw], ignore_index=True)
print(f"Joined Shape: {gdsc_raw_join.shape}")

assert gdsc_raw_join[gdsc_raw_join.index.duplicated()].shape[0] == 0
assert gdsc_raw_join.shape[0] == gdsc1_raw.shape[0] + gdsc2_raw.shape[0]

gdsc_raw_join.head(5)

/var/folders/kv/l0jzxgbj1kggff_kd35bfzkw0000gn/T/ipykernel_60135/877327064.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  gdsc1_raw = pd.read_csv(f'{PATH_TO_GDSC_SCREENING_DATA}{GDSC1_RAW_FILE}', header=0)


File `GDSC1_public_raw_data_25Feb20.csv` took 14.26997 seconds to import. It has shape (5837703, 18)
File `GDSC2_public_raw_data_25Feb20.csv` took 16.06028 seconds to import. It has shape (6646430, 18)
Joined Shape: (12484133, 18)


,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,CELL_ID,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SEEDING_DENSITY,DRUGSET_ID,ASSAY,DURATION,POSITION,TAG,DRUG_ID,CONC,INTENSITY
0,Sanger_GDSC1,100541,1765,2010-04-18T23:00:00Z,NaN,2415,365,924238,K5,250.0,505_a_5,a,3.0,1,B,NaN,NaN,26022
1,Sanger_GDSC1,100541,1765,2010-04-18T23:00:00Z,NaN,2415,365,924238,K5,250.0,505_a_5,a,3.0,2,B,NaN,NaN,20491
2,Sanger_GDSC1,100541,1765,2010-04-18T23:00:00Z,NaN,2415,365,924238,K5,250.0,505_a_5,a,3.0,3,UN-USED,NaN,NaN,181946
3,Sanger_GDSC1,100541,1765,2010-04-18T23:00:00Z,NaN,2415,365,924238,K5,250.0,505_a_5,a,3.0,4,UN-USED,NaN,NaN,213387
4,Sanger_GDSC1,100541,1765,2010-04-18T23:00:00Z,NaN,2415,365,924238,K5,250.0,505_a_5,a,3.0,5,UN-USED,NaN,NaN,166452


In [14]:
# Join both tables.
cols_to_join_on = ['CELL_LINE_NAME', 'DRUG_ID']
gdsc_ic50_raw_join = gdsc_ic50s_join.merge(gdsc_raw_join,
                                           on=cols_to_join_on,
                                           how='left',
                                           suffixes=['_ic50', '_raw'])
print(f"Shape after joining both GDSC tables (raw and ic50): {gdsc_ic50_raw_join.shape}")

if gdsc_ic50_raw_join.shape[0] > gdsc_ic50s_join.shape[0]:
    print(f"""There are multiple {cols_to_join_on} entries in the raw GDSC table which match with the IC50 GDSC table.
        number of rows after left join   : {gdsc_ic50_raw_join.shape[0]}
        number of rows in the IC50 table : {gdsc_ic50s_join.shape[0]}
        number of rows in the raw table  : {gdsc_raw_join.shape[0]}
    """)

gdsc_ic50_raw_join.head(5)

Shape after joining both GDSC tables (raw and ic50): (5707271, 35)
There are multiple ['CELL_LINE_NAME', 'DRUG_ID'] entries in the raw GDSC table which match with the IC50 GDSC table.
        number of rows after left join   : 5707271
        number of rows in the IC50 table : 446146
        number of rows in the raw table  : 12484133
    


,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID_ic50,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,...,MASTER_CELL_ID,COSMIC_ID_raw,SEEDING_DENSITY,DRUGSET_ID,ASSAY,DURATION,POSITION,TAG,CONC,INTENSITY
0,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,...,49,683665,NaN,410_a_9,s,3.0,14,L1-D1-S,2.000,544404
1,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,...,49,683665,NaN,410_a_9,s,3.0,15,L1-D2-S,1.000,599897
2,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,...,49,683665,NaN,410_a_9,s,3.0,16,L1-D3-S,0.500,587627
3,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,...,49,683665,NaN,410_a_9,s,3.0,17,L1-D4-S,0.250,597272
4,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,...,49,683665,NaN,410_a_9,s,3.0,18,L1-D5-S,0.125,594490


In [17]:
COLS_TO_EXCLUDE = [
    # From IC50 table.
    'NLME_RESULT_ID',
    'NLME_CURVE_ID',
    'TCGA_DESC',
    'PUTATIVE_TARGET',
    'PATHWAY_NAME',
    'COMPANY_ID',
    'MIN_CONC',
    'MAX_CONC',
    # From RAW table.
    'WEBRELEASE',
    'RESEARCH_PROJECT',
    'BARCODE', 
    'SCAN_ID', 
    'DATE_CREATED',
    'SCAN_DATE',
    'COSMIC_ID_raw',
    'DRUGSET_ID',
    'ASSAY',
    'TAG',
    'SEEDING_DENSITY',
    'DURATION'
]

gdsc_sparsed_cols = gdsc_ic50_raw_join[list(set(gdsc_ic50_raw_join.columns) - set(COLS_TO_EXCLUDE))]
gdsc_sparsed_cols = gdsc_sparsed_cols.rename(columns={'COSMIC_ID_ic50': 'COSMIC_ID'})
print(f"Shape after excluding non-interesting columns: {gdsc_sparsed_cols.shape}")
gdsc_sparsed_cols.head(5)

Shape after excluding non-interesting columns: (5707271, 15)


,CONC,SANGER_MODEL_ID,CELL_LINE_NAME,Z_SCORE,MASTER_CELL_ID,DRUG_ID,AUC,LN_IC50,RMSE,CELL_ID,INTENSITY,POSITION,DRUG_NAME,COSMIC_ID,DATASET
0,2.000,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,544404,14,Erlotinib,683665,GDSC1
1,1.000,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,599897,15,Erlotinib,683665,GDSC1
2,0.500,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,587627,16,Erlotinib,683665,GDSC1
3,0.250,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,597272,17,Erlotinib,683665,GDSC1
4,0.125,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,594490,18,Erlotinib,683665,GDSC1


In [20]:
# Since we don't want duplicates with the same LN_IC50 information in our data we will only take the rows with the maximal concentrations.
DUPLICATE_GROUP_COLS = ['MASTER_CELL_ID', 'DRUG_ID', 'LN_IC50']
gdsc_without_duplicates = gdsc_sparsed_cols[gdsc_sparsed_cols.groupby(DUPLICATE_GROUP_COLS)['CONC'].transform('max') == gdsc_sparsed_cols['CONC']]
gdsc_without_duplicates = gdsc_without_duplicates[gdsc_without_duplicates.groupby(DUPLICATE_GROUP_COLS)['INTENSITY'].transform('max') == gdsc_without_duplicates['INTENSITY']]
print(f"Shape after removing duplicates: {gdsc_without_duplicates.shape}")
gdsc_without_duplicates.head(5)

Shape after removing duplicates: (446521, 15)


,CONC,SANGER_MODEL_ID,CELL_LINE_NAME,Z_SCORE,MASTER_CELL_ID,DRUG_ID,AUC,LN_IC50,RMSE,CELL_ID,INTENSITY,POSITION,DRUG_NAME,COSMIC_ID,DATASET
0,2.0,SIDM00636,MC-CAR,-0.189576,49,1,0.982114,2.395685,0.022521,3137,544404,14,Erlotinib,683665,GDSC1
9,2.0,SIDM00265,ES3,0.508635,1342,1,0.984816,3.140923,0.031840,2366,404197,14,Erlotinib,684055,GDSC1
27,2.0,SIDM00263,ES5,1.284229,610,1,0.985693,3.968757,0.026052,2368,797378,14,Erlotinib,684057,GDSC1
45,2.0,SIDM00269,ES7,0.088760,71,1,0.972699,2.692768,0.110056,2371,377574,14,Erlotinib,684059,GDSC1
54,2.0,SIDM00203,EW-11,-0.111820,231,1,0.944462,2.478678,0.087011,2375,638065,14,Erlotinib,684062,GDSC1


In [21]:
# Save the GDSC table to a file.
gdsc_without_duplicates.to_pickle(f'{PATH_TO_SAVE_DATA_TO}gdsc_base_v2.pkl')